In [ ]:
import sys
sys.path.append("..") # for sibling import

import compyute as cp

# Example 5.1

### Language Model: Bigram model

The goal of this model is to be able to generate text that is similar to the training data using a single character to predict the next one.

### Step 1: Prepare data
The dataset can be downloaded from https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt. Afterwards, it needs to be placed it into the */data* directory.

In [ ]:
with open("../data/tinyshakespeare.txt", "r") as f:
    data = f.read()

### Step 2: Tokenization
To train a neural network on text, it needs to be represented by numerical values. For this reason a tokenizer is used. To build a vocabulary of tokens, here the `CharacterTokenizer` is used.

In [ ]:
from compyute.preprocessing.text import CharacterTokenizer

tknzr = CharacterTokenizer()
tknzr.vocab_size

Every character is assigned an integer token.

In [ ]:
tknzr.vocab

Tokenizers can be used to encode text and decode token ids. Here, the string "Hello World!" is encoded and afterwards represented by the respective token ids.

In [ ]:
string = "Hello World!"
encoded_string = tknzr.encode(string)
encoded_string

A tensor of token ids can also be decoded again to get the original text.

In [ ]:
[tknzr.decode([i]) for i in encoded_string]

In [ ]:
tknzr.decode(encoded_string)

To use the data to train a model, it needs to be encoded.

In [ ]:
data_enc = tknzr.encode(data)
data_enc[:20]

### Step 3: Build dataset
Next up, we need to build the training dataset.

In [ ]:
X = cp.Tensor([data_enc[i] for i in range(len(data_enc) - 1)])
y = cp.Tensor([data_enc[i + 1] for i in range(len(data_enc) - 1)])

X_shuffle, idx = cp.random.shuffle(X)
y_shuffle = y[idx]

n = int(len(X) * 0.9)

X_train = X_shuffle.int()[:n, None]
y_train = y_shuffle.int()[:n]
X_val = X_shuffle.int()[n:, None]
y_val = y_shuffle.int()[n:]

print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_val.shape=}")
print(f"{y_val.shape=}")

### Step 4: Build the neural network structure

An `Embedding` layer is used to assign each token an n-dimensional vector. The vector's components are then learned and updated during the training process.

In [ ]:
import compyute.nn as nn

model = nn.Sequential(
    nn.Embedding(tknzr.vocab_size, tknzr.vocab_size),
    nn.Flatten()
)

In [ ]:
model.summary(input_shape=(1,), input_dtype="int32")

### Step 5: Train the model

Since there are usually quite a large number of classes (=tokens) in language models, the training process can be slower.

In [ ]:
from compyute.nn.trainer import losses, metrics, optimizers, Trainer
from compyute.nn.trainer.callbacks import ProgressBar

trainer = Trainer(
    model=model,
    optimizer=optimizers.AdamW(lr=1e-2),
    loss=losses.CrossEntropy(),
    metric=metrics.Accuracy(),
    callbacks=[ProgressBar()]
)

In [ ]:
trainer.train(X_train, y_train, epochs=1)

### Step 6: Analyze Results

In [ ]:
# !pip install scikit-learn matplotlib

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# get numpy array of embedding table
embs = model.modules[0].w.cpu().data

# reduce dimensions to 2 to make
tsne = TSNE(random_state=0).fit_transform(embs)

# plot results
plt.figure(figsize=(8, 8))
plt.scatter(x=tsne[:,0], y=tsne[:,1], alpha=0.5, s=100)
plt.axis("off")
for i in range(len(tsne)):
    char = tknzr.decode(cp.Tensor([i]))
    plt.text(x=tsne[i,0]-0.1, y=tsne[i,1]-0.15, s=char)

### Step 7: Generate text
To see the model in action, it is given a starting character (here token 1). Then it is used to generate $n$ characters using previous charactes as input. The better the model is trained, the more sensical the output will be. This can take quite a lot of training though.

In [ ]:
context = cp.ones((1, 1,), dtype="int32") # use 1 as startig context
n = 1000

for _ in range(n):
    pred = model(context).squeeze() # predict following character using the current context
    probs, _ = cp.nn.functional.softmax(pred)
    index = cp.random.multinomial(
        x=pred.shape[-1],
        p=probs,
        shape=(1,)
    ) # choose a character from prediction
    print(tknzr.decode([index.item()]), end="")
    context = context.append(index[None, :], axis=1).int() # append predicted character to context
    context = context[:, 1:] # set new context